In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
from scipy import signal
from scipy.interpolate import CubicSpline

In [ ]:
%matplotlib qt

In [ ]:
EXPERIMENT = 'A04703'

In [ ]:
data = pd.read_csv(f'T:/Team/Szewczyk/Data/2024-09-12/{EXPERIMENT}_two_electrodes.dat')

In [ ]:
sos = signal.butter(20, 0.1, fs=0.5 ,output='sos')
data['el1_filt'] = signal.sosfiltfilt(sos, data.el1)
data['el2_filt'] = signal.sosfiltfilt(sos, data.el2)

In [ ]:
cs1 = CubicSpline(data.t, data.el1_filt)
cs2 = CubicSpline(data.t, data.el2_filt)
new_time = np.arange(0, data.t.iloc[-1], 0.01)
fit = pd.DataFrame({
    't'     : new_time,
    'el1'   : cs1(new_time),
    'el2'   : cs2(new_time),
})

In [ ]:
peaks1 = signal.find_peaks(fit.el1[40000:], distance=500)[0]+40000
peaks2 = signal.find_peaks(fit.el2[40000:], distance=500)[0]+40000

In [ ]:
peaks1 = np.delete(peaks1, 41)

In [ ]:
plt.figure()
plt.plot(fit.t, fit.el1, label = 'electrode 1')
plt.plot(fit.t, fit.el2, label = 'electrode 2')
plt.title(f'{EXPERIMENT} waveforms')
plt.xlabel('t [s]')
plt.ylabel(r'$\xi$')
plt.legend()
# plt.scatter(fit.t[peaks1], fit.el1[peaks1])

In [ ]:
csp1 = CubicSpline(peaks1/100, range(peaks1.shape[0]))
fit['phase1'] = csp1(fit.t)
csp2 = CubicSpline(peaks2/100, range(peaks2.shape[0]))
fit['phase2'] = csp2(fit.t)

In [ ]:
plt.figure()
plt.title(f'Phase difference {EXPERIMENT}')
plt.xlabel('t [s]')
plt.ylabel(r'$\Delta \phi$')
plt.axhline(0, ls='--')
plt.plot(fit.t, fit.phase1 - fit.phase2)

In [ ]:
plt.figure()
plt.plot(fit.phase1.iloc[peaks2]%1)

In [ ]:
plt.plot(fit.t, fit.el1)
plt.plot(fit.t, fit.phase1%1*5+70)

In [ ]:
plt.figure()
plt.title(f'LA synchronisation {EXPERIMENT}')

plt.xlabel('t [s]')
plt.ylabel('T [s]')


plt.plot(fit.t[peaks1][1:], np.diff(fit.t[peaks1]), label='electrode 1')
plt.plot(fit.t[peaks2][1:], np.diff(fit.t[peaks2]), label='electrode 2')
plt.legend()